In [1]:
import numpy as np

from PIL import Image

from build123d import *
from OCP import BRepBuilderAPI, gp

In [4]:
im = Image.open('hth3_background.jpg')
imarr = np.array(im).mean(axis=-1)
imarr = (imarr - imarr.min())/np.ptp(imarr)

In [5]:
width = 50
height = 20
image_extent = width-height

In [6]:
base = extrude(Rectangle(width, width), height)
base

Part at 0x799934b6eae0, label(), #children(0)

In [7]:
def make_occ_mesh_faces(imgarr, rescale_xy=None):
    faces = []

    if rescale_xy is None:
        isc = jsc = 1
    else:
        isc = (imgarr.shape[0]-1) / float(rescale_xy)
        jsc = (imgarr.shape[1]-1) / float(rescale_xy)

    for i in range(imgarr.shape[0]-1):
        for j in range(imgarr.shape[1]-1):
            verts = ((i/isc, j/jsc, imgarr[i,j]), 
                     (i/isc, (j+1)/jsc, imgarr[i, j+1]), 
                     ((i+1)/isc, (j+1)/jsc, imgarr[i+1, j+1]), 
                     ((i+1)/isc, j/jsc, imgarr[i+1, j]))
            p0, p1, p2, p3 = (gp.gp_Pnt(*c) for c in verts)

            for ps in ((p0, p1, p2), (p0,p2,p3)):
                polygon_builder = BRepBuilderAPI.BRepBuilderAPI_MakePolygon(*ps, Close=True)
                face_builder = BRepBuilderAPI.BRepBuilderAPI_MakeFace(polygon_builder.Wire())
                face = face_builder.Face()
                faces.append(face)

    return faces

def polyface_from_verts(verts):
    
    polygon_builder = BRepBuilderAPI.BRepBuilderAPI_MakePolygon()
    for vs in verts:
        polygon_builder.Add(gp.gp_Pnt(*vs))
    polygon_builder.Close()
    face_builder = BRepBuilderAPI.BRepBuilderAPI_MakeFace(polygon_builder.Wire())
    return face_builder.Face()

def make_mesh_sides(imgarr, rescale_xy=None):
    if rescale_xy is None:
        isc = jsc = 1
    else:
        isc = (imgarr.shape[0]-1) / float(rescale_xy)
        jsc = (imgarr.shape[1]-1) / float(rescale_xy)

    iedge = (imgarr.shape[0]-1) / isc
    jedge = (imgarr.shape[1]-1) / jsc

    verts = [(i/isc, 0, imgarr[i, 0]) for i in range(imgarr.shape[0])]
    verts = [(0,0,0)] + verts + [(iedge, 0, 0)]
    iplus = polyface_from_verts(verts)
    
    verts = [(i/isc, jedge, imgarr[i, imgarr.shape[1]-1]) for i in range(imgarr.shape[0])]
    verts = [(0,jedge,0)] + verts + [(iedge, jedge, 0)]
    iminus = polyface_from_verts(verts)

    verts = [(0, j/jsc, imgarr[0, j]) for j in range(imgarr.shape[1])]
    verts = [(0,0,0)] + verts + [(0, jedge, 0)]
    jplus = polyface_from_verts(verts)  
    
    verts = [(iedge, j/jsc, imgarr[imgarr.shape[0]-1, j]) for j in range(imgarr.shape[1])]
    verts = [(iedge,0,0)] + verts + [(iedge, jedge, 0)]
    jminus = polyface_from_verts(verts)    

    bottom = polyface_from_verts([(iedge, 0, 0), 
                                  (iedge, jedge, 0), 
                                  (0, jedge, 0), 
                                  (0, 0, 0)])


    return [iplus, iminus, jplus, jminus, bottom]

def make_mesh_box(imgarr, rescale_xy=None, make_solid=True):
    mesh_faces = make_occ_mesh_faces(imgarr, rescale_xy)
    side_faces = make_mesh_sides(imgarr, rescale_xy)

    ofaces = mesh_faces + side_faces

    if make_solid:
        faces = [Face(f) for f in ofaces]
        return Solid(Shell(faces))
    else:
        return ofaces

In [8]:
meshbox = make_mesh_box(imarr[::10, ::10]*image_extent, width)
combined = Pos(-width/2, -width/2, height)*meshbox + base
combined

Part at 0x799933fbe840, label(), #children(0)